# 만약, csv 파일로 raw 데이터로 끌고 온다면 여기부터 시작

In [ ]:
import pandas as pd
import numpy as np
import pickle

def save_pkl(data, path):
    with open(path, 'wb') as ww:
        pickle.dump(data, ww)
        
def load_pkl(path):
    with open(path, 'rb') as rr:
        data = pickle.load(rr)
    return data

In [ ]:
# london = pd.read_pickle("london.pkl")

In [ ]:
# station = pd.read_csv("london_stations.csv")

## Case에 맞게 전처리

In [ ]:
# 전처리 코드




## Co_occurence matrix 만드는 코드

```python

df_matrix = df_original[['end_station_id', 'start_station_id']]

co_matrix = pd.crosstab(df_matrix['start_station_id'], df_matrix['end_station_id'])


```

## Community detection

### 노드와 에지 만들기

In [ ]:
from copy import deepcopy

In [ ]:
def get_edge(from_, to_):
    
    row_boolean = co_matrix['start_station_id'] == from_
    
    value = co_matrix[row_boolean][str(float(to_))].values[0]
    
    return value

In [ ]:
nodes = [id_ for id_ in co_matrix['start_station_id']]

edges = [[from_, to_, get_edge(from_, to_)] for from_ in nodes for to_ in nodes]

node_and_edge = {'nodes' : nodes, 'edges' : edges}

In [ ]:
df = pd.DataFrame(node_and_edge['edges'], columns =['from', 'to', 'value'], dtype=int)

df_edges = deepcopy(df)

df_edges = df_edges.rename(columns = {'from' : 'source', 'to' : 'target'})
df_edges

In [ ]:
edges = df_edges[['source','target']].values.tolist()
weights = [float(l) for l in df_edges.value.values.tolist()]

In [ ]:
G = nx.Graph(directed=True)
G.add_edges_from(edges)

In [ ]:
for cnt, a in enumerate(G.edges(data=True)):
    G.edges[(a[0], a[1])]['weight'] = weights[cnt]

In [ ]:
# 참고 : https://lsjsj92.tistory.com/587

fig = plt.figure(figsize=(20,10))

# compute the best partition
partition = community_louvain.best_partition(G)

# draw the graph
pos = nx.spring_layout(G)
# color the nodes according to their partition
cmap = cm.get_cmap('viridis', max(partition.values()) + 1)
nx.draw_networkx_nodes(G, pos, partition.keys(), node_size=40,
                       cmap=cmap, node_color=list(partition.values()))
nx.draw_networkx_edges(G, pos, alpha=0.5)
plt.show()

### node per class

In [ ]:
node_per_class = {i : [] for i in range(8)}
node_per_class

In [ ]:
for i in nodes:
    for cls in range(8):
        if partition[int(i)] == cls:
            node_per_class[cls].append(i)